In [1]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import os
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from math import pi
from helpers.spot import *
from helpers.misc import *
from time import sleep

In [2]:
# constants
ON_REPEAT = '37i9dQZF1Epfk0vNhDMCzw'
REPEAT_REWIND = '37i9dQZF1EpCchDHzfQcYR'
HIPHOP_NL = '0Fa3x3yr0duQJStzGpnH7w'

In [3]:
# seting up
client_id = os.environ.get('SPOT_CLIENT_ID')
secret = os.environ.get('SPOT_SECRET')

auth_manager = SpotifyClientCredentials(client_id, secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [4]:
playlist_tracks = get_playlist_tracks(sp, HIPHOP_NL)
ids = get_ids(playlist_tracks)

In [19]:
# getting track meta and features

track_metas = []
track_features = []
splitted_ids = list(split_list(ids, 50))

d1 = sp.tracks(splitted_ids[1])

for track in d1['tracks']:
    name, explicit, popularity = track['name'], track['explicit'], track['popularity']
    

Nawoord True 39
Quarantaine Sessie #1 False 43
Arme Zielen True 29
Terug Naar Toen True 38
Wat Is Echt True 0
Loyaal Zijn True 26
Alleen Soms True 40
Paranoia False 40
Allemaal Op Brieven False 41
Onderweg True 40
Winterdagen False 43
De Prijs True 42
Niet Genoeg True 41
Nog Steeds False 40
Blauwe Lichten True 46
Alles Of Niks True 46
Vastberaden True 48
Mijn Story True 47
Studiosessie 304 - 101Barz True 52
Gordelweg True 52
Punchline Killer True 28
Nooit Meer Terug False 30
Maskerade True 26
Maniac True 43
Til 'Em Op True 49
Moest Al Lang True 36
Kort Lontje True 47
Geen Zin Soms True 36
MANSORY True 57
Het Leven Is Een Trip True 54
Dag Tot Dag True 28
Wil je weten hoe et voelt False 36
SEASONS - SE1EP1: Lijpe True 47
Slapeloze Nachten False 64
Hella Papers True 38
Op Die Rap True 28
Watskeburt?! True 57
Sterrenstof True 64
Noodzaak False 0
Puinhoop False 17
Mijn Pad - Live in de Ziggo Dome False 9
Uit elkaar - Live in de Ziggo Dome False 6
Nu zie je (feat. Lijpe) False 34
Liever Alle

In [71]:
len(track_metas)

407

In [ ]:
# data parsing
needed_metas = ['name', 'explicit', 'popularity']
needed_features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
dict_data = dict((key, []) for key in (needed_metas+needed_features))

for meta, features in zip(track_metas, track_features):
    m = [meta[key] for key in needed_metas]
    f = [features[key] for key in needed_features]
    mf = m+f
    for key, value in zip(dict_data.keys(), mf):
        dict_data[key].append(value)

data = pd.DataFrame.from_dict(dict_data)
num_data = data.iloc[:,2:]

# save data
data.to_csv(path_or_buf=f'{playlist["name"]}.csv')

In [ ]:
num_data.loc[:] = MinMaxScaler().fit_transform(num_data.loc[:])

# plot size
fig=plt.figure(figsize=(12,8))

# convert column names into a list
categories=list(num_data.columns)
# number of categories
N=len(categories)

# create a list with the average of all features
value=list(num_data.mean())

# repeat first value to close the circle
# the plot is a circle, so we need to "complete the loop"
# and append the start value to the end.
value+=value[:1]
# calculate angle for each category
angles=[n/float(N)*2*pi for n in range(N)]
angles+=angles[:1]

# plot
plt.polar(angles, value)
plt.fill(angles,value,alpha=0.3)

# plt.title('Discovery Weekly Songs Audio Features', size=35)

plt.xticks(angles[:-1],categories, size=15)
plt.yticks(color='grey',size=15)
plt.show()